## 0. Imports and Setup

In [1]:
import mysql.connector
import pandas as pd
import json

In [2]:
database = mysql.connector.connect(
    user='root', 
    password='password',
    host='127.0.0.1', 
    port=3307,
    database="TPCH",
)

In [3]:
cursor = database.cursor()

## 1. Query Data

In [4]:
sql_reader = open("queries/test_queries.sql")
queries = sql_reader.read().split(";")
sql_reader.close()

In [5]:
print(queries[0])

-- using 1697822052 as a seed to the RNG



select
	l_returnflag,
	l_linestatus,
	sum(l_quantity) as sum_qty,
	sum(l_extendedprice) as sum_base_price,
	sum(l_extendedprice * (1 - l_discount)) as sum_disc_price,
	sum(l_extendedprice * (1 - l_discount) * (1 + l_tax)) as sum_charge,
	avg(l_quantity) as avg_qty,
	avg(l_extendedprice) as avg_price,
	avg(l_discount) as avg_disc,
	count(*) as count_order
from
	lineitem
where
	l_shipdate <= date '1998-12-01' - interval '83' day
group by
	l_returnflag,
	l_linestatus
order by
	l_returnflag,
	l_linestatus


In [6]:
pd.read_sql(queries[0], database)

/var/folders/k2/zns3zzl52ngb09pgfbbnb3pm0000gn/T/ipykernel_60315/2788461944.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  pd.read_sql(queries[0], database)


,l_returnflag,l_linestatus,sum_qty,sum_base_price,sum_disc_price,sum_charge,avg_qty,avg_price,avg_disc,count_order
0,A,F,380456.0,5.323482e+08,5.058224e+08,5.261659e+08,25.575155,35785.709307,0.050081,14876
1,N,F,8971.0,1.238480e+07,1.179826e+07,1.228249e+07,25.778736,35588.509684,0.047759,348
2,N,O,746245.0,1.046263e+09,9.942616e+08,1.034122e+09,25.456949,35691.564384,0.049925,29314
3,R,F,381449.0,5.345944e+08,5.079965e+08,5.285242e+08,25.597168,35874.006533,0.049828,14902


## 2. Evaluate Performance

In [7]:
cursor.execute("EXPLAIN FORMAT='JSON'" + queries[0])
estimated_performance = json.loads(cursor.fetchall()[0][0])
estimated_performance

{'query_block': {'select_id': 1,
  'cost_info': {'query_cost': '26559.77'},
  'ordering_operation': {'using_filesort': False,
   'grouping_operation': {'using_temporary_table': True,
    'using_filesort': True,
    'cost_info': {'sort_cost': '19862.35'},
    'table': {'table_name': 'lineitem',
     'access_type': 'ALL',
     'rows_examined_per_scan': 59593,
     'rows_produced_per_join': 19862,
     'filtered': '33.33',
     'cost_info': {'read_cost': '4711.19',
      'eval_cost': '1986.23',
      'prefix_cost': '6697.43',
      'data_read_per_join': '7M'},
     'used_columns': ['l_orderkey',
      'l_linenumber',
      'l_quantity',
      'l_extendedprice',
      'l_discount',
      'l_tax',
      'l_returnflag',
      'l_linestatus',
      'l_shipdate'],
     'attached_condition': "(`TPCH`.`lineitem`.`l_shipdate` <= <cache>((DATE'1998-12-01' - interval '83' day)))"}}}}}

In [8]:
cursor.execute("EXPLAIN ANALYZE " + queries[0].replace("lineitem", "lineitem USE INDEX()"))
actual_performance = cursor.fetchall()[0][0]
print(actual_performance)

-> Sort: lineitem.l_returnflag, lineitem.l_linestatus  (actual time=146..146 rows=4 loops=1)
    -> Table scan on <temporary>  (actual time=146..146 rows=4 loops=1)
        -> Aggregate using temporary table  (actual time=146..146 rows=4 loops=1)
            -> Filter: (lineitem.l_shipdate <= <cache>((DATE'1998-12-01' - interval '83' day)))  (cost=6697 rows=19862) (actual time=0.387..78.2 rows=59440 loops=1)
                -> Table scan on lineitem  (cost=6697 rows=59593) (actual time=0.3..66.7 rows=60175 loops=1)



## 3. Modify Schema

In [9]:
cursor.execute("CREATE INDEX lineitem_returnflag_linestatus ON lineitem (l_returnflag, l_linestatus)")

In [10]:
cursor.execute("EXPLAIN FORMAT='JSON'" + queries[0])
estimated_performance = json.loads(cursor.fetchall()[0][0])
estimated_performance

{'query_block': {'select_id': 1,
  'cost_info': {'query_cost': '6697.43'},
  'ordering_operation': {'using_filesort': False,
   'grouping_operation': {'using_filesort': False,
    'table': {'table_name': 'lineitem',
     'access_type': 'index',
     'possible_keys': ['lineitem_returnflag_linestatus'],
     'key': 'lineitem_returnflag_linestatus',
     'used_key_parts': ['l_returnflag', 'l_linestatus'],
     'key_length': '8',
     'rows_examined_per_scan': 59593,
     'rows_produced_per_join': 19862,
     'filtered': '33.33',
     'cost_info': {'read_cost': '4711.19',
      'eval_cost': '1986.23',
      'prefix_cost': '6697.43',
      'data_read_per_join': '7M'},
     'used_columns': ['l_orderkey',
      'l_linenumber',
      'l_quantity',
      'l_extendedprice',
      'l_discount',
      'l_tax',
      'l_returnflag',
      'l_linestatus',
      'l_shipdate'],
     'attached_condition': "(`TPCH`.`lineitem`.`l_shipdate` <= <cache>((DATE'1998-12-01' - interval '83' day)))"}}}}}

In [11]:
cursor.execute("EXPLAIN ANALYZE " + queries[0])
actual_performance = cursor.fetchall()[0][0]
print(actual_performance)

-> Group aggregate: sum(lineitem.l_quantity), sum(lineitem.l_extendedprice), sum((lineitem.l_extendedprice * (1 - lineitem.l_discount))), sum(((lineitem.l_extendedprice * (1 - lineitem.l_discount)) * (1 + lineitem.l_tax))), avg(lineitem.l_quantity), avg(lineitem.l_extendedprice), avg(lineitem.l_discount), count(0)  (cost=8684 rows=3) (actual time=60.2..167 rows=4 loops=1)
    -> Filter: (lineitem.l_shipdate <= <cache>((DATE'1998-12-01' - interval '83' day)))  (cost=6697 rows=19862) (actual time=0.42..139 rows=59440 loops=1)
        -> Index scan on lineitem using lineitem_returnflag_linestatus  (cost=6697 rows=59593) (actual time=0.413..132 rows=60175 loops=1)



## 4. Ignore Index

In [12]:
cursor.execute("EXPLAIN FORMAT='JSON'" + queries[0].replace("lineitem", "lineitem USE INDEX()"))
estimated_performance = json.loads(cursor.fetchall()[0][0])
estimated_performance

{'query_block': {'select_id': 1,
  'cost_info': {'query_cost': '26559.77'},
  'ordering_operation': {'using_filesort': False,
   'grouping_operation': {'using_temporary_table': True,
    'using_filesort': True,
    'cost_info': {'sort_cost': '19862.35'},
    'table': {'table_name': 'lineitem',
     'access_type': 'ALL',
     'rows_examined_per_scan': 59593,
     'rows_produced_per_join': 19862,
     'filtered': '33.33',
     'cost_info': {'read_cost': '4711.19',
      'eval_cost': '1986.23',
      'prefix_cost': '6697.43',
      'data_read_per_join': '7M'},
     'used_columns': ['l_orderkey',
      'l_linenumber',
      'l_quantity',
      'l_extendedprice',
      'l_discount',
      'l_tax',
      'l_returnflag',
      'l_linestatus',
      'l_shipdate'],
     'attached_condition': "(`TPCH`.`lineitem`.`l_shipdate` <= <cache>((DATE'1998-12-01' - interval '83' day)))"}}}}}

In [13]:
cursor.execute("EXPLAIN ANALYZE " + queries[0].replace("lineitem", "lineitem USE INDEX()"))
actual_performance = cursor.fetchall()[0][0]
print(actual_performance)

-> Sort: lineitem.l_returnflag, lineitem.l_linestatus  (actual time=90.6..90.6 rows=4 loops=1)
    -> Table scan on <temporary>  (actual time=90.6..90.6 rows=4 loops=1)
        -> Aggregate using temporary table  (actual time=90.6..90.6 rows=4 loops=1)
            -> Filter: (lineitem.l_shipdate <= <cache>((DATE'1998-12-01' - interval '83' day)))  (cost=6697 rows=19862) (actual time=0.0689..46.7 rows=59440 loops=1)
                -> Table scan on lineitem  (cost=6697 rows=59593) (actual time=0.0641..39.1 rows=60175 loops=1)

